In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, RNN, Layer
from tensorflow.keras.optimizers import Adam


In [48]:
# Load the arrays from .npy files
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')
print(f'Shape of y_train: {y_train.shape}')
print(f'Shape of y_test: {y_test.shape}')

Shape of X_train: (8816, 911, 39)
Shape of X_test: (2204, 911, 39)
Shape of y_train: (8816,)
Shape of y_test: (2204,)


In [14]:
unique_labels, counts = np.unique(y_train, return_counts=True)
label_counts = dict(zip(unique_labels, counts))
print(f'Number of different labels in y_train: {len(unique_labels)}')
print(f'Counts of each label in y_train: {label_counts}')

Number of different labels in y_train: 2
Counts of each label in y_train: {np.float64(0.0): np.int64(34059), np.float64(1.0): np.int64(11353)}


In [43]:
class LTCCell(Layer):
    def __init__(self, units, ode_unfolds=6, **kwargs):
        super(LTCCell, self).__init__(**kwargs)
        self.units = units
        self.ode_unfolds = ode_unfolds
        self.state_size = units

    def build(self, input_shape):
        self.input_dim = input_shape[-1]
        # Trainable parameters
        self.W = self.add_weight(shape=(self.input_dim + self.units, self.units),
                                 initializer='glorot_uniform',
                                 name='W')
        self.bias = self.add_weight(shape=(self.units,),
                                    initializer='zeros',
                                    name='bias')
        self.tau = self.add_weight(shape=(self.units,),
                                   initializer='ones',
                                   name='tau')
        super(LTCCell, self).build(input_shape)

    def call(self, inputs, states):
        prev_state = states[0]
        concatenated = tf.concat([inputs, prev_state], axis=1)
        dt = 0.01  # Time step
        for _ in range(self.ode_unfolds):
            dh = (-prev_state + tf.nn.tanh(tf.matmul(concatenated, self.W) + self.bias)) / tf.nn.softplus(self.tau)
            prev_state += dt * dh
        return prev_state, [prev_state]


In [44]:
def build_lnn_model(input_shape, units, num_classes):
    inputs = Input(shape=input_shape)
    ltc_cell = LTCCell(units=units, ode_unfolds=6)
    rnn_layer = RNN(ltc_cell, return_sequences=False)
    x = rnn_layer(inputs)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # (time_steps, features)
units = 128
num_classes=4
model = build_lnn_model(input_shape, units, num_classes)


In [45]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [46]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 911, 39)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn (RNN)                       │ (None, 128)            │        21,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,148 (86.52 KB)

 Trainable params: 22,148 (86.52 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping, lr_reduction]
)


In [ ]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


In [ ]:
# Predict classes
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification report
target_names = ['angry', 'happy', 'sad', 'neutral']
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
model.save('emotion_recognition_model.h5')
print("Model saved to emotion_recognition_model.h5")